<a href="https://colab.research.google.com/github/fardsnog2/ML_vvsu/blob/main/lab5/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Линейная регрессия

In [6]:
!pip install datasets seaborn
import pandas as Панда
import numpy as Нампай
import seaborn as снс
import matplotlib.pyplot as плт
from sklearn.model_selection import train_test_split as обучающая_тестовая

In [7]:
#ПРОСТИТЕ МЕНЯ
Вывод = print()
Целое = int
Строка = str
Дробная = float
Словарь = dict
Список = list
Множество = set
Прочитать_данные_эксель = Панда.read_excel


##Выбор и исключение переменных

Загрузите набор данных из файла ```Regressions datasets``` в соответствии с вашим вариантом (о смене датасета - индивидуально)

In [ ]:
import pandas

In [9]:
дата_сет = Прочитать_данные_эксель('/content/moviedb (4).xls')
дата_сет

,Unnamed: 0,title,release,runtime,budget,revenue,lang,genre1,genre2,country1,...,writer,studio1,studio2,keyword1,keyword2,keyword3,keyword4,keyword5,keyword6,rating
0,0,"""Vegeta"" (Shrek)",NaN,99.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,1,'93: Del Soul,2021-07-18 00:00:00,15.0,250.0,NaN,en,детектив,триллер,NaN,...,Riley Lockhart,Elkman Productions,NaN,sports car,psychological stress,axe murder,horror filmmaking,1990s,carsploitation,2.0
2,2,'Catch Me If You Can': Behind the Camera,2003-05-06 00:00:00,17.0,NaN,NaN,NaN,документальный,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.7
3,3,'Harry Potter': Behind the Magic,2005-11-19 00:00:00,48.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0
4,4,"'Oh, Miss Jones!': The Very Best of Leonard Ro...",1996-08-16 00:00:00,52.0,NaN,NaN,en,документальный,NaN,GB,...,NaN,Telegram,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10277,10277,Белфаст,2021-11-12 00:00:00,98.0,NaN,30000000.0,en,драма,NaN,GB,...,Kenneth Branagh,Northern Ireland Screen,TKBC,working class,black and white,"belfast, north ireland",1960s,NaN,NaN,7.2
10278,10278,Финч,2021-11-04 00:00:00,115.0,2500000.0,500000000.0,en,фантастика,драма,US,...,Ivor Powell,Amblin Entertainment,ImageMovers,robot,journey,humanity,post-apocalyptic,NaN,NaN,8.1
10279,10279,Под гипнозом,2021-10-27 00:00:00,88.0,NaN,NaN,en,триллер,драма,CA,...,Richard D'Ovidio,The Long Game,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.1
10280,10280,Последняя дуэль,2021-10-13 00:00:00,152.0,100000000.0,27000000.0,en,боевик,драма,GB,...,NaN,20th Century Studios,Scott Free Productions,based on novel or book,"normandy, france",based on true story,knight,duel,historical,7.6


### Предварительный анализ

1. Выбрать целевую переменную Y (Y - числоая переменная для задачи регрессии)
2. С помощью EDA (см практика) отобрать признаки, которые влияют на Y (можно определить гарфически, можно числами).
3. Обработать пропуски, при необходимости почистить от выбросов (тут важное замечание - не все выбросы это плохо)
4. Обосновать выбор переменных для исследования (корреляция, различия по классам, и так далее)

P.S. Важное замечание, слишком уникальные признаки (имена людей, названия город, ID юзера или чего-то еще - не берем, так как они не влияют на целевую переменную, а если и влияет, то это случайно)

In [ ]:
"""
Код здесь
"""
У = дата_сет['rating']


### Трансформация набора данных

(OneHotEncode для категориальных переменных)

(Standard or MinMax or бининг или другое преобразование для числовых)

P.S. Целевую переменную не преобразовываем, оставляем как есть


In [ ]:
"""
Код здесь
"""

### Разделение данных

Для начала, разделить данные на X и Y

Разбить данные на тестовую и обучающую выборки

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
"""
Код здесь
"""

### Обучение
#### 1 Регрессия
- Обучить линейную регрессию на тренировочных данных (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)
- Сдлеать .predict() для X_train, X_test
- Рассчитать метрики качества MAE, MSE, MAPE, R2
- Вывести и написать выводы о коэфициентах регрессии
- Написать формулу регрессии

Сделать выводы о переобучении модели (см метрики на тестовой выборке) сделать выводы о метриках на тестовом наборе

#### 2 Ближайшие соседи
- Обучить KNN (https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor)
- Сдлеать .predict() для X_train, X_test
- Рассчитать метрики качества MAE, MSE, MAPE, R2

Сделать выводы о переобучении модели (см метрики на тестовой выборке) сделать выводы о метриках на тестовом наборе

In [ ]:
"""
Код здесь
"""

##Задание для гениев*

Удачи 😈

В практике есть примеры реализации ```linear_regression``` и ```knn_regressor```, но есть нюанс... Сейчас ни linear_regression, ни knn_regressor не поддерживают сохранения состояния для последующего возможного использования.

Необходимо - реализовать два класса LinearRegression и KNNRegression (примеры обучения моделек есть). В этих классах обязатльно должны быть два метода
```
.fit(X: np.array, y: np.array) -> None - обучить модель и сохранить внутреннее состояние модели
.predict(X) -> np.array - предсказать y на основе X
```
При инициализации класса - передавать параметры модели в момент инициализации. Например:

```
model = LinearRegression(**params)

model.fit(X_train, y_train)
model.predict(X_test)
```

В качестве тестов можно использовать KNN и LinearRegression из sklearn, результат работы ваших объектов должен примерно совпадать с результатами работы объектов из sklearn

В результате работы сделать сравнение с теми же данными в sklearn

In [ ]:
"""
Результат работы
"""